In [5]:
import os, json
import pandas as pd
import fnmatch
import boto3
from botocore.config import Config
import sagemaker

from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

sagemaker_session = sagemaker.Session()

s3 = boto3.resource('s3')

# Destination Bucket
bucket = 'piiv'
role = sagemaker.get_execution_role()

In [6]:
from sagemaker.pytorch import PyTorch

# Criando o Estimator
pytorch_estimator = PyTorch('transfer_learning.py',
                            role=role,
                            instance_type='ml.m5.large',
                            instance_count=1,
                            framework_version='1.5.0',
                            py_version='py3',
                           )

In [7]:
#bucket_uri = 's3://' + bucket + '/' + prefix
bucket_uri = 's3://' + bucket
pytorch_estimator.fit({'training': bucket_uri})

2021-12-03 20:29:51 Starting - Starting the training job...
2021-12-03 20:30:16 Starting - Launching requested ML instancesProfilerReport-1638563391: InProgress
......
2021-12-03 20:31:17 Starting - Preparing the instances for training.........
2021-12-03 20:32:37 Downloading - Downloading input data...
2021-12-03 20:33:17 Training - Downloading the training image...
2021-12-03 20:33:37 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-12-03 20:33:32,826 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-12-03 20:33:32,841 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-12-03 20:33:32,852 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-12-03 20:33:39,073 sagemaker_pytorch_container.training INFO     Invoking user training scr

In [8]:
predictor = pytorch_estimator.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

-----!

In [10]:
from PIL import Image
import torch
from torchvision import transforms
import numpy as np

loader = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])        

def image_loader(image_name):
    image = Image.open(image_name)
    image = loader(image).float()
    image = image.unsqueeze(0) 
    return image

s3 = boto3.resource('s3')
bucket = s3.Bucket('piiv')
object = bucket.Object('010_0009.jpg') #key
response = object.get()
img = response['Body']

image = image_loader(img)

objects_category = ['01-beer-mug','02-coffee-mug','03-teapot','04-wine-bottle']

response = predictor.predict(image)
output = torch.exp(torch.tensor(response))
index = np.argmax(output)
print("Result --> label: " + objects_category[index] + " | probability: " + str(output[0][index]))

Result --> label: 01-beer-mug | probability: tensor(0.6802, dtype=torch.float64)
